<h1>Tarea 1, parte 1</h1>
<h2>Pregunta 1</h2>

NGC 121 es un cúmulo globular que se encuentra en la dirección de la constelación de Tucana (RA 00 26 49.0; DEC -71 32 10), en la Pequeña Nube de Magallanes. Fue descubierto en 1835 por John Herschel.
Su módulo de distancia es de $(M-m)=18.94$, que corresponde a una distancia de $61KPc$. Es poco extendido, con un radio de $53.7Pc$ y su edad se estima en 10 mil millones de años.

Fuentes:
<ol>
<li><a href="http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?objname=NGC+121&extend=no&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES#ObjNo1">IPAC</a></li>
<li><a href="https://www.nasa.gov/content/goddard/oldest-cluster/#.V6fI4N9ythE">NASA</a></li>
<li><a href="http://simbad.u-strasbg.fr/simbad/sim-id?Ident=%403279874&Name=NGC+++121&submit=display+all+measurements#lab_meas">Simbad</a></li>
</ol>
<hr>
<h2>Pregunta 2</h2><br>

Se utilizarán los dos resultados al consultar la Hubble Legacy Archive por imágenes de nivel 2 obtenidas usando ACS/HRC. ACS es la <em>Advanced Camera for Surveys</em>, que contiene tres cámaras; la HRC, <em>High Resolution Camera</em> (0.025''/px), es una de ellas (actualmente se encuentra inoperable).

Las imágenes fueron tomadas en las bandas correspondientes a los filtros amplios (150nm de ancho aproximadamente) F555W, centrado en el verde de 530.8nm; y F814W, centrado en 802.4nm, infrarrojo. Los filtros del sistema fotométrico más cercanos serían justamente el <strong>V</strong>, centrado en 551nm y el <strong>I</strong>, centrado en 806nm.

Cada archivo FITS incluye las extensiones:
<ol>
 <li><strong>PRIMARY</strong>, que contiene información en el <em>header</em> pero no contiene imagen.</li>
 <li><strong>SCI</strong> (science), que contiene la imagen de ciencia.</li>
 <li><strong>WHT</strong> (weight), que contiene una imagen que representa un mapa con el peso relativo de cada píxel según el inverso de la varianza.</li>
 <li><strong>CTX</strong> (context), que contiene una imagen que representa un mapa de cuáles imágenes (antes de combinarlas) contribuyen en cada píxel de la imagen final combinada.</li>
 <li><strong>HDRTAB</strong> (header table), que guarda registro de los valores que cambian en el header de una imagen combinada tras agregar una nueva imagen.</li>
 <li><strong>EXP</strong> (exposure), que indica el peso por píxel dado por el tiempo de exposición que tuvo su imagen original.</li>
</ol>

Se requiere guardar cada imagen de ciencia y el mapa de pesos en un archivo separado. Para ello, se utiliza el módulo astropy.io.fits:

In [1]:
from astropy.io import fits

f555w = fits.open('hst_10396_a4_acs_hrc_f555w_drz.fits')
f814w = fits.open('hst_10396_a4_acs_hrc_f814w_drz.fits')

f555w[1].writeto('sci_f555w_n121.fits',clobber=True)
f555w[2].writeto('invar_f555w_n121.fits',clobber=True)

f814w[1].writeto('sci_f814w_n121.fits',clobber=True)
f814w[2].writeto('invar_f814w_n121.fits',clobber=True)

f555w.close()
f814w.close()

Por último, se puede comprobar que las imágenes están bien alineadas con ayuda de <em>DS9</em> si se eligen algunas estrellas y se compara su posición.
<hr>
<h2>Pregunta 3</h2>
<br>
Para esta pregunta se utilizó SExtractor (SE). Los parámetros de base se tomaron de las configuraciones por defecto de SE (archivo default.sex) pero se cambiaron las opciones de acuerdo a lo requerido, para cada imagen, según información disponible en la web del Space Telescope Science Institute:
 <ul>
  <li><a href="http://www.stsci.edu/hst/acs/documents/handbooks/cycle19/c04_detector4.html#372434">HRC; ganancia, saturación, etc</a></li>
  <li><a href="http://www.stsci.edu/hst/acs/analysis/zeropoints/old_page/localZeropoints">Tablas de zeropoints</a></li>
 </ul>
Y también en <a href="http://iopscience.iop.org/article/10.1086/444553/pdf">Sirianni et al. (2005)</a>.

Se ejecutó de la siguiente manera:

<tt>sextractor sci_f555w_n121.fits -c f555w.sex</tt><br>
<tt>sextractor sci_f814w_n121.fits -c f814w.sex</tt>

Como resultado se obtuvieron catálogos que incluyen las columnas <strong>MAG_APER, MAGERR_APER, ALPHA_J2000</strong> y <strong>DELTA_J2000</strong>.

Se obtuvieron <strong>6144</strong> objetos para la imagen en el filtro f555w y <strong>4801</strong> para la imagen en el filtro f814w. Los archivos de configuración y los catálogos obtenidos están en este repositorio.
<hr>
<h2>Pregunta 4</h2>
Una vez obtenidos los catálogos para ambos filtros, se realiza una comparación y <em>match</em> entre ambos para obtener las fuentes comunes. Esto se hace con el módulo de python <strong>astropy.coordinates</strong>:

In [4]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
import numpy as np
import os

# Se cargan listas con RA y DEC para cada imagen
RAf555w = np.loadtxt('f555w.cat',usecols=(3,))
DEf555w = np.loadtxt('f555w.cat',usecols=(4,))

RAf814w = np.loadtxt('f814w.cat',usecols=(3,))
DEf814w = np.loadtxt('f814w.cat',usecols=(4,))

# Match por parte de astropy
catalog = SkyCoord(ra=RAf555w*u.degree, dec=DEf555w*u.degree)  
c = SkyCoord(ra=RAf814w*u.degree, dec=DEf814w*u.degree)  
idx = c.match_to_catalog_sky(catalog)

# Del catalogo f555w.cat se extraen las filas que indica el match
matches = list(idx[0])
f555w = np.loadtxt('f555w.cat')
f814w = np.loadtxt('f814w.cat')
out = []

j = 0
for i in matches:
	out.append(np.concatenate([f555w[i],f814w[j]]))
	j = j+1

# Salida a archivo
np.savetxt('n121_match.cat',out,
	fmt='%d\t%.4f\t%.4f\t%.7f\t%.7f\t%d\t%.4f\t%.4f\t%.7f\t%.7f',
	header='f555wN\tf555wMAG\tf555wMAGERR\tf555wALPHA\tf555wDELTA\tf814wN\tf814wMAG\tf814wMAGERR\tf814wALPHA\tf814wDELTA',
	footer='END')


De esta forma se obtiene un catálogo de <strong>4801</strong> objetos que se logran emparejar en ambas imágenes. Si se observa que de la imagen en filtro f814w se obtuvieron exactamente 4801 fuentes, entonces se evidencia que para cada estrella en dicha imagen se encontró un par en el filtro f555w. Esto demuestra que las imágenes están bien alineadas. Sin embargo, a distintas fuentes del filtro f814w se les emparejó con la misma fuente en f555w por parte de astropy. Las coincidencias entre ambos catálogos sin contar estas duplicidades son de alrededor del 92%. 

El siguiente paso es calibrar las magnitudes. Se utiliza el sistema <em>VEGAMAG</em>, en donde la magnitud de Vega en cualquier filtro se define como <strong>0</strong>. Se ajusta el <strong>zeropoint</strong> en este sistema, que se define como la magnitud de un objeto que produce una cuenta por segundo para un tiempo de exposición determinado. Como el Zeropoint fue un parámetro de entrada para SE, ya se calibró esta parte.

También hay que considerar la <strong>aperture correction</strong> para la HRC. Dado que al hacer fotometría se utilizan aperturas pequeñas para mejorar la relación señal a ruido (SNR), solo una parte de la energía priveniente de la fuente logra medirse. Se incluye una corrección para "recuperar" las cuentas de diferencia que se obtendrían de utilizar una apertura "infinita". Estos ajustes están presentes en <a href="http://iopscience.iop.org/article/10.1086/444553/pdf">Sirianni et al. (2005)</a>. Para aplicar esta correción (dada en unidades de magnitud) basta con adicionar el valor de ajuste a la magnitud que ya se tenía, para cada filtro. Se utilizó $0.24$ para el filtro f555w y $0.452$ para el f814w, para una apertura de $0.025''/px \times 6px = 0.15''$. 

Por último, hay que tener en cuenta el <strong>reddening</strong>, que se refiere a la absorción y dispersión de luz por parte del polvo interestelar, que afecta más a la luz azul que a la roja, por lo que las fuentes se enrojecen. Esto depende, etonces, de la banda en que se observe y del objeto. Otra vez de Sirianni, se toman los valores magnitud de $3.180$ y $1.803$ (que corresponden a objetos del tipo G2) para los filtros f555w y f814w, respectivamente. Estos números se restan de las magnitudes anteriores, de modo que se obtiene una magnitud que representa un objeto más brillantes de lo que se observa.


In [1]:
from astropy.io import ascii
import matplotlib
import matplotlib.pyplot as plt

# Genera plot
tbl = ascii.read('n121_match.cat')
plt.scatter((tbl["f555wMAG"]+0.24) - (tbl["f814wMAG"]+0.452) +  (3.180-1.803), (tbl["f555wMAG"]+0.24-3.180),c='black',s=1)
plt.xlim(0, 3)
plt.ylim(reversed(plt.ylim(14,22.5)))
plt.xlabel("$F555W-F814W$", fontsize=20)
plt.ylabel("$F555W$", fontsize=20)
plt.savefig('cmd_n121', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

<img src="cmd_n121.png">